In [1]:
import numpy as np
from scapy.all import *
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [2]:
# concat = pd.concat([pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap1.csv"), 
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap2.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap3.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap4.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap5.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap6.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap7.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap8.csv"),
#                    pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/pcap9.csv")], ignore_index=True)

In [3]:
# concat.to_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/swat_pcap_features.csv")

In [4]:
bad_cols = [0, 15, 16, 20, 24, 25, 26, 27, 30] # Index, SIntPkt, DIntPkt, IdleTime, sDSb, sTtl, SAppBytes, DAppBytes, sTos
use_cols = [i for i in range(0, 30) if i not in bad_cols]

swat = pd.read_csv("/home/knel/datasets/SWaT/SWaT.A6_Dec 2019/pcap/swat_pcap_features.csv", usecols=use_cols).to_numpy()

In [5]:
sports_index, dports_index, protocols_index = 1, 2, 14

In [6]:
# remove arp protocol rows (blank ports)
swat = swat[np.where(swat[:, 14] != "arp")]

# remove igmp protocol rows (blank ports)
swat = swat[np.where(swat[:, 14] != "igmp")]

# remove man protocol rows (some weird artifact from Argus I dunno)
swat = swat[np.where(swat[:, 14] != "man")]

In [7]:
len(swat)

2658168

In [25]:
# one-hot encode Sports
enc_sports = OneHotEncoder()
enc_sports.fit(swat[:, sports_index].reshape(-1, 1))
# sports_one_hot = enc_sports.transform(swat[:, sports_index].reshape(-1, 1)).toarray()

# need to insert new column for every category from the one-hot encoder
# this dataset is going to have SO MANY COLUMNS


# one-hot encode Dports
enc_dports = OneHotEncoder()
enc_dports.fit(swat[:, dports_index].reshape(-1, 1))
# dports_one_hot = enc_dports.transform(swat[:, dports_index].reshape(-1, 1)).toarray()

# one-hot encode Proto
enc_proto = OneHotEncoder()
enc_proto.fit(swat[:, protocols_index].reshape(-1, 1))
# dports_one_hot = enc_proto.transform(swat[:, protocols_index].reshape(-1, 1)).toarray()

OneHotEncoder()

In [26]:
type(swat[0][1]), swat[0]

(str,
 array([4.999887, '56327', '44818', 3910, 3913, 373788, 334332,
        597922.3125, 534807.3125, 781.817688, 782.417664, 976, 978,
        19.985681, 'tcp', 4.999887, 0.0, 4.999887, 4.999887, 4.999887, 0.0,
        4.999887], dtype=object))

In [37]:
sports_one_hot_flat = []
dports_one_hot_flat = []
proto_one_hot_flat = []

swat_split = np.array_split(swat, 100_000)
# swat_split = np.array_split(swat[:100], 10)

for split in swat_split:
    sports_one_hot_flat.append(enc_sports.transform(split[:, sports_index].reshape(-1, 1)).toarray())
    dports_one_hot_flat.append(enc_dports.transform(split[:, dports_index].reshape(-1, 1)).toarray())
    proto_one_hot_flat.append(enc_proto.transform(split[:, protocols_index].reshape(-1, 1)).toarray())

In [ ]:
sports_one_hot_flat[0], np.where(sports_one_hot_flat[1][0] == 1), np.where(sports_one_hot_flat[:][0] == 1)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (array([1572]),),
 (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  array([14753, 14129,   140,  7213,  1571, 14718, 15796,  1553,  1551,
           140])))

In [32]:
test = []

test.append(enc_sports.transform(swat[:10, sports_index].reshape(-1, 1)).toarray())


test2 = np.asarray([])
test2 = np.append(test2, enc_sports.transform(swat[:10, sports_index].reshape(-1, 1)).toarray())

np.where(test[0][0] == 1)

(array([14753]),)

In [154]:
np.array_equal(test[0], test2.reshape(10, 23030))

True

In [7]:
attack_pkt_list = []
for pkt in swat:
    timestamp = pkt.time
    relative_time = timestamp - TIME_ZERO
    # check if packet from within attack timeframes
    if (relative_time >= 900 and relative_time <= 4500) or (relative_time >= 8700 and relative_time <= 12300):
        attack_pkt_list.append(pkt)

In [8]:
len(attack_pkt_list)

0

In [9]:
for i in range(20):
    print(swat[i].time - TIME_ZERO)

0.000000
0.000002
0.000003
0.000004
0.000006
0.000007
0.000008
0.000010
0.000113
0.000114
0.000269
0.000271
0.000272
0.000587
0.000588
0.000918
0.000919
0.001267
0.001268
0.001421


In [18]:
pkt_list = []
for pkt in swat:
    pkt_list.append(pkt)

src_ips = []
dst_ips = []
for pkt in pkt_list:
    try:
        src_ips.append(pkt[IP].src)
        dst_ips.append(pkt[IP].dst)
    except IndexError:
        continue;

src_ips_set = set(src_ips)
dst_ips_set = set(dst_ips)

print(src_ips_set)
print(dst_ips_set)

{'192.168.1.20', '192.168.1.200', '192.168.1.31', '153.254.86.163', '192.168.1.40', '192.168.1.60', '192.168.1.130', '192.168.1.118', '192.168.1.90', '192.168.1.176', '192.168.1.51', '192.168.1.11', '192.168.1.230', '192.168.1.177', '192.168.1.61', '192.168.1.219', '10.10.10.2', '192.168.1.10', '91.189.94.4', '192.168.1.30', '192.168.1.100', '118.201.255.203', '192.168.1.99', '192.168.1.179', '192.168.1.50', '192.168.1.234', '192.168.1.183', '192.168.1.41', '192.168.1.178', '192.168.1.175', '192.168.1.201', '192.168.1.21', '192.168.1.220'}
{'192.168.1.20', '192.168.1.200', '153.254.86.163', '192.168.1.40', '192.168.1.60', '192.168.1.130', '224.0.0.252', '239.192.2.63', '192.168.1.176', '192.168.1.11', '192.168.1.230', '192.168.1.177', '192.168.50.128', '192.168.1.255', '192.168.1.219', '255.255.255.255', '239.192.4.191', '192.168.1.10', '91.189.94.4', '192.168.1.30', '192.168.1.100', '118.201.255.203', '10.10.10.7', '239.192.5.255', '192.168.1.99', '239.192.8.127', '239.255.255.250', '